# imports

In [24]:
import numpy as np
import pandas as pd

# Generate new fitting script from template

In [22]:
#There are three burst where not all detectors are on
#modify the fitting script by hand, since the data grouping will be different

def fit_script(HEDfname=None,KW1fname=None,KW2fname=None,GBMn1fname=None,GBMn2fname=None,GBMb1fname=None,outfname=None):
    
    fnames = [HEDfname,KW1fname,KW2fname,GBMn1fname,GBMn2fname,GBMb1fname,outfname]
    if all(isinstance(fname, str) for fname in fnames) == False:
        return(print("input must be string"))
    
    with open('fitting.xcm',"r") as f:
        lines = f.readlines() #198 lines of code
        for i in np.arange(0,len(lines)):
            lines[i] = lines[i].replace("HED_PHA_1.fits",HEDfname)
            lines[i] = lines[i].replace("KW201227_1_gr20.pha",KW1fname)
            lines[i] = lines[i].replace("KW201227_2_gr20.pha",KW2fname)
            lines[i] = lines[i].replace("GRB201227A_n4_gr20.pha",GBMn1fname)
            lines[i] = lines[i].replace("GRB201227A_n8_gr20.pha",GBMn2fname)
            lines[i] = lines[i].replace("GRB201227A_b0_gr20.pha",GBMb1fname)
            
    with open(outfname+".xcm","w") as out:
        for line in lines:
            out.write(line)

In [59]:
fit_script(HEDfname="bla",KW1fname="blaa",KW2fname="blaaa",GBMn1fname="la",GBMn2fname="laa",GBMb1fname="laaa",outfname="fooo")

In [23]:
#GRB 190206A don't have GBM data, so i can't use the fit_script above
#I can go in an replace HED and KW files, but then i need to remove the GBM grouping and modeling that includes GBM

# Get spectral parameters from log file
    -'Current model list' starts from index 144
    -'\n' contained in every line

In [2]:
with open('template.log','r') as f:
    logLines = f.readlines()
    #looking for this line: #Model constant<1>*cflux<2>*ngrbep<3> Source No.: 1   Active/On
    for l in logLines:
        if l == '#Model constant<1>*cflux<2>*ngrbep<3> Source No.: 1   Active/On':
            print(l)

In [15]:
for i,l in enumerate(logLines):
    if l == '#Current model list:\n':
        print('model list start at index: ', i)
        modList = logLines[i:]
        break

model list start at index:  144


In [ ]:
#TODO: load the spec parameters for all the datagroups
#remove '\n' from line

list

In [57]:
clean_modList = []
for l in modList:
    l = l.replace('\n','')
    l = l.replace('#','')
    clean_modList.append(l)

In [115]:
keys = {}
for i,l in enumerate(clean_modList):
    if 'Data group:' in l:
        key = l.strip()
        keys[key] = i
startingIndecies = np.fromiter(keys.values(), dtype=int)
n_dataGroups = len(startingIndecies)

if np.all(np.diff(startingIndecies)) == True:
    size_dataGroup = np.diff(np.fromiter(keys.values(), dtype=int))[0]
else:
    print('varying length of datagroups')

In [108]:
startingIndecies,n_dataGroups

(array([ 6, 16, 26, 36]), 4)

In [159]:
spec_data = {}
for key,value in keys.items():
    newTargetModel = []
    targetModel = clean_modList[value+1:value+size_dataGroup]
    for par in targetModel:
        newpar = par.strip()
        newpar = newpar.split()
        del newpar[1]
        newTargetModel.append(newpar)
    spec_data[key] = newTargetModel

In [160]:
spec_data

{'Data group: 1': [['1', 'constant', 'factor', '1.27737', '+/-', '0.127300'],
  ['2', 'cflux', 'Emin', 'keV', '10.0000', 'frozen'],
  ['3', 'cflux', 'Emax', 'keV', '1.00000E+04', 'frozen'],
  ['4', 'cflux', 'lg10Flux', 'cgs', '-4.18518', '+/-', '3.00161E-02'],
  ['5', 'ngrbep', 'alpha', '-0.185890', '+/-', '7.28931E-02'],
  ['6', 'ngrbep', 'beta', '-3.44617', '+/-', '0.512888'],
  ['7', 'ngrbep', 'epeak', 'keV', '922.399', '+/-', '58.2440'],
  ['8', 'ngrbep', 'enorm', 'keV', '100.000', 'frozen'],
  ['9', 'ngrbep', 'norm', '1.00000', 'frozen']],
 'Data group: 2': [['10', 'constant', 'factor', '1.00000', 'frozen'],
  ['11', 'cflux', 'Emin', 'keV', '10.0000', '=', 'p2'],
  ['12', 'cflux', 'Emax', 'keV', '1.00000E+04', '=', 'p3'],
  ['13', 'cflux', 'lg10Flux', 'cgs', '-4.18518', '=', 'p4'],
  ['14', 'ngrbep', 'alpha', '-0.185890', '=', 'p5'],
  ['15', 'ngrbep', 'beta', '-3.44617', '=', 'p6'],
  ['16', 'ngrbep', 'epeak', 'keV', '922.399', '=', 'p7'],
  ['17', 'ngrbep', 'enorm', 'keV', '100.